# Webscrapping

To extract data from websites,  we can use the **read_html** function to directly get DataFrames from a **url**. 

In [1]:
import pandas as pd
url = "https://en.wikipedia.org/wiki/World_population"
dataframe_list = pd.read_html(url, flavor='bs4')
display(dataframe_list[5])

,Rank,Country,Population,Area(km2),Density(pop/km2)
0,1,Singapore,5704000,710,8033
1,2,Bangladesh,172240000,143998,1196
2,3,Palestine,5266785,6020,847
3,4,Lebanon,6856000,10452,656
4,5,Taiwan,23604000,36193,652
5,6,South Korea,51781000,99538,520
6,7,Rwanda,12374000,26338,470
7,8,Haiti,11578000,27065,428
8,9,Netherlands,17690000,41526,426
9,10,Israel,9480000,22072,429


However, sometimes the server refuses to authorize the request **(HTTP Error 403: Forbidden )**.

In [2]:
url= 'https://www.worldometers.info/world-population'
dataframe_list = pd.read_html(url, flavor='bs4')

HTTPError: HTTP Error 403: Forbidden

To overcome this issue, we can scrape data from HTML tables into a Dataframe using BeautifulSoup

In [4]:
from bs4 import BeautifulSoup # module for web scrapping.
import requests  #  module to download a web page

In [5]:
url= 'https://www.worldometers.info/world-population'
data  = requests.get(url).text
soup = BeautifulSoup(data,"html.parser")

In [6]:
#find all html tables in the web page
tables = soup.find_all('table')

In [7]:
# we can see how many tables were found by checking the length of the tables list
number_of_tables= len(tables)
number_of_tables

5

**Function to get the columns names**

In [8]:
def get_column_names(my_table):
    columns= my_table.findAll('th') 
    columns_name = [x.text.strip() for x in columns] 
    return columns_name
    

**Function to create the dataframe corresponding to a table**

In [9]:
def create_dataframe(table):
    columns_name=get_column_names(table)
    df = pd.DataFrame(columns=columns_name)
    for row in table.tbody.find_all("tr"):
        col = row.find_all("td")
        if (col != []):
            my_data = [x.text.strip() for x in col] # List comprehension
            my_data_columns=zip(columns_name, my_data)
            df = df.append(dict(my_data_columns), ignore_index=True)         
    return df

**Loop over all tables in the websitet**: create a dataframe for each table and display it. Adittionally, we can save the table in a csv format. If the table has a caption, the csv file will be save with the caption name. Otherwise it will be saved with the name 'tableX'

In [10]:
my_list=[]
for index,table in enumerate(tables):
        table_index = index
        title=table.find('caption')
        if title==None:
            save_name='table_'+ str(table_index)
        else:
            save_name= title.text.strip()
        print(save_name)
        my_df=create_dataframe(table)
        my_list.extend([my_df])
        display(my_df)
        # Save df to csv
        my_df.to_csv(save_name+'.csv')  



table_0


,Year (July 1),Population,Yearly % Change,Yearly Change,Median Age,Fertility Rate,Density (P/Km²),Urban Pop %,Urban Population
0,2018,"7,631,091,040",1.10 %,"83,232,115",29.8,2.51,51,55.3 %,"4,219,817,318"
1,2017,"7,547,858,925",1.12 %,"83,836,876",29.8,2.51,51,54.9 %,"4,140,188,594"
2,2016,"7,464,022,049",1.14 %,"84,224,910",29.8,2.51,50,54.4 %,"4,060,652,683"
3,2015,"7,379,797,139",1.19 %,"84,594,707",30,2.52,50,54.0 %,"3,981,497,663"
4,2010,"6,956,823,603",1.24 %,"82,983,315",28,2.58,47,51.7 %,"3,594,868,146"
5,2005,"6,541,907,027",1.26 %,"79,682,641",27,2.65,44,49.2 %,"3,215,905,863"
6,2000,"6,143,493,823",1.35 %,"79,856,169",26,2.78,41,46.7 %,"2,868,307,513"
7,1995,"5,744,212,979",1.52 %,"83,396,384",25,3.01,39,44.8 %,"2,575,505,235"
8,1990,"5,327,231,061",1.81 %,"91,261,864",24,3.44,36,43.0 %,"2,290,228,096"
9,1985,"4,870,921,740",1.79 %,"82,583,645",23,3.59,33,41.2 %,"2,007,939,063"


table_1


,Year (July 1),Population,Yearly % Change,Yearly Change,Median Age,Fertility Rate,Density (P/Km²),Urban Pop %,Urban Population
0,2020,"7,794,798,739",1.10 %,"83,000,320",31,2.47,52,56.2 %,"4,378,993,944"
1,2025,"8,184,437,460",0.98 %,"77,927,744",32,2.54,55,58.3 %,"4,774,646,303"
2,2030,"8,548,487,400",0.87 %,"72,809,988",33,2.62,57,60.4 %,"5,167,257,546"
3,2035,"8,887,524,213",0.78 %,"67,807,363",34,2.70,60,62.5 %,"5,555,833,477"
4,2040,"9,198,847,240",0.69 %,"62,264,605",35,2.77,62,64.6 %,"5,938,249,026"
5,2045,"9,481,803,274",0.61 %,"56,591,207",35,2.85,64,66.6 %,"6,312,544,819"
6,2050,"9,735,033,990",0.53 %,"50,646,143",36,2.95,65,68.6 %,"6,679,756,162"


table_2


""
0


table_3


,#,Region,Population(2020),YearlyChange,NetChange,Density(P/Km²),Land Area(Km²),Migrants(net),Fert.Rate,Med.Age,UrbanPop %,WorldShare
0,1,Asia,"4,641,054,775",0.86 %,"39,683,577",150,"31,033,131","-1,729,112",2.2,32,0 %,59.5 %
1,2,Africa,"1,340,598,147",2.49 %,"32,533,952",45,"29,648,481","-463,024",4.4,20,0 %,17.2 %
2,3,Europe,"747,636,026",0.06 %,"453,275",34,"22,134,900","1,361,011",1.6,43,0 %,9.6 %
3,4,Latin America and the Caribbean,"653,962,331",0.9 %,"5,841,374",32,"20,139,378","-521,499",2,31,0 %,8.4 %
4,5,Northern America,"368,869,647",0.62 %,"2,268,683",20,"18,651,660","1,196,400",1.8,39,0 %,4.7 %
5,6,Oceania,"42,677,813",1.31 %,"549,778",5,"8,486,460","156,226",2.4,33,0 %,0.5 %


table_4


,#,Country (or dependency),Population(2020),YearlyChange,NetChange,Density (P/Km²),Land Area (Km²),Migrants(net),Fert.Rate,Med.Age,UrbanPop %,WorldShare
0,1,China,"1,439,323,776",0.39 %,"5,540,090",153,"9,388,211","-348,399",1.69,38,60.8 %,18.5 %
1,2,India,"1,380,004,385",0.99 %,"13,586,631",464,"2,973,190","-532,687",2.2402,28,35 %,17.7 %
2,3,United States,"331,002,651",0.59 %,"1,937,734",36,"9,147,420","954,806",1.7764,38,82.8 %,4.2 %
3,4,Indonesia,"273,523,615",1.07 %,"2,898,047",151,"1,811,570","-98,955",2.3195,30,56.4 %,3.5 %
4,5,Pakistan,"220,892,340",2 %,"4,327,022",287,"770,880","-233,379",3.55,23,35.1 %,2.8 %
...,...,...,...,...,...,...,...,...,...,...,...,...
230,231,Montserrat,"4,992",0.06 %,3,50,100,,N.A.,N.A.,9.6 %,0 %
231,232,Falkland Islands,"3,480",3.05 %,103,0,"12,170",,N.A.,N.A.,66 %,0 %
232,233,Niue,"1,626",0.68 %,11,6,260,,N.A.,N.A.,46.4 %,0 %
233,234,Tokelau,"1,357",1.27 %,17,136,10,,N.A.,N.A.,0 %,0 %
